#### Observe that we can get news of a day with the link format:
http://news.ltn.com.tw/list/newspaper/politics/20181231
#### We need a list of date string in the date range we want.

In [10]:
from datetime import datetime, timedelta

start_date = "2018-11-15"
stop_date = "2018-11-30"

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y%m%d'))
    start = start + timedelta(days=1)


#### Now write a function to parse the HTML response, return the data we want(title, content, ...etc).

In [11]:
import requests
from bs4 import BeautifulSoup as bs

In [12]:
def process_document(document, date):
    
    nodes = document.select('ul.list > li')
    data = list()

    for li in nodes:

        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
    return data

#### Crawl over the news on the site, store the data in variable "all_data" .

In [13]:
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://news.ltn.com.tw/list/newspaper/politics/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data

start crawling : 20181115
start crawling : 20181116
start crawling : 20181117
start crawling : 20181118
start crawling : 20181119
start crawling : 20181120
start crawling : 20181121
start crawling : 20181122
start crawling : 20181123
start crawling : 20181124
start crawling : 20181125
start crawling : 20181126
start crawling : 20181127
start crawling : 20181128
start crawling : 20181129
start crawling : 20181130


#### Check the result

In [14]:
all_data[0:5]

[{'date': '2018-11-15',
  'title': '侯補繳稅 蘇酸沒欠為何要補',
  'link': 'http://news.ltn.com.tw/news/politics/paper/1246946',
  'content': '〔記者李雅雯、何玉華、陳心瑜、彭琬馨／綜合報導〕民進黨新北市長候選人蘇貞昌繼續抨擊國民黨市長候選人侯友宜在文大大群館逃漏稅、新莊購地案等，對於侯友宜解釋是主動補繳稅，蘇貞昌質疑說，「沒有欠稅為什麼要補繳？這不是自打嘴巴嗎？」他表示選舉真的是照妖鏡，經過選舉才發現侯友宜這麼有錢，有大房子還逃漏稅，在副市長任內買新莊土地等待都更，他呼籲選民檢驗候選人的理財態度。民進黨立委段宜康昨也指責侯友宜「公然說謊」，指又昱「若沒有欠，為何要補？若有補，表示有欠（稅）」，呼籲侯友宜儘快出面釐清事實。蘇貞昌則指出，他參選不是為了爭取一個職位，而是爭取為新北市改變的機會。過去擔任台北縣長時，開發超過一千公頃的土地，沒有購買一寸土地，他請市民選擇「在各地開發土地而不買地」的人，或「在位子上經營自家財產、買地大賺錢」的人。對於侯友宜反擊蘇貞昌也繼承一千坪的土地，蘇回應說，他在屏東的農地是父親過世後繼承而來，地上種的是廿年的樹，是用來養樹、種樹的農地，一坪一萬元；侯友宜在新莊買地一坪近五十萬元，而且是在副市長任內購地，還沒當選就開始買地、養地，將來做了首長還得了，上位者若這樣做，底下的人可能就更誇張了。蘇貞昌的發言人黃韋鈞說，根據最新土地公告現值，蘇繼承的農地，每平方公尺僅五千元，但侯家在新莊的土地是住宅區，每平方公尺十三．六萬元，離新莊捷運站直線距離僅四百五十公尺，是精華地段住宅用地，兩者無法比擬。對於侯友宜指「下跪催票」是民進黨的招數，蘇貞昌怒批說「我聽了很憤怒」，盧修一受盡國民黨欺凌、坐過苦牢，在罹癌化療那麼痛苦的最後，下跪拜託台北縣民要選對人；他非常感謝盧修一，對於這樣的民主前輩，侯怎麼好意思嘲笑？侯當年對盧噴辣椒水、強押上車，如今不知反省，竟然諷刺、污辱民主前輩，「我真的很不屑」。',
  'tags': []},
 {'date': '2018-11-15',
  'title': '大群館、購地案遭圍攻 侯︰「東西南北廠」蜂擁而至',
  'link': 'http://news.ltn.com.tw/news/politi

#### Save as pkl file

In [15]:
import pickle

with open('data/liberty_times_copy_1.pkl', 'wb') as f:
    pickle.dump(all_data, f)

#### Turn it into pandas dataframe

In [16]:
import pandas as pd
pd.DataFrame(all_data)[['date', 'title', 'link', 'content', 'tags']].head()

,date,title,link,content,tags
0,2018-11-15,侯補繳稅 蘇酸沒欠為何要補,http://news.ltn.com.tw/news/politics/paper/124...,〔記者李雅雯、何玉華、陳心瑜、彭琬馨／綜合報導〕民進黨新北市長候選人蘇貞昌繼續抨擊國民黨市長...,[]
1,2018-11-15,大群館、購地案遭圍攻 侯︰「東西南北廠」蜂擁而至,http://news.ltn.com.tw/news/politics/paper/124...,〔記者賴筱桐、李雅雯／新北報導〕民進黨新北市長候選人蘇貞昌抨擊國民黨市長候選人侯友宜，在文化...,[]
2,2018-11-15,嘉市議會「保護」涉賄副議長／法部︰仍可境管 避免郭明賓逃亡,http://news.ltn.com.tw/news/politics/paper/124...,〔記者吳政峰／台北報導〕嘉義地檢署懷疑嘉義市議會副議長郭明賓涉賄選，發函嘉義市議會，要求同意...,[]
3,2018-11-15,凱聚炒股案被提訊 傅崐萁哭求回花蓮,http://news.ltn.com.tw/news/politics/paper/124...,〔記者楊國文／台北報導〕有「花蓮王」稱號的前花蓮縣長傅崐萁，因合機炒股案被判八月，已入獄一個...,[]
4,2018-11-15,投票前夕網上拉票 不能超過深夜12點,http://news.ltn.com.tw/news/politics/paper/124...,〔記者李欣芳／台北報導〕九合一選舉投票前一晚十時競選活動結束後，能否進行網路直播，或在臉書等...,[]
